In [1]:
import json

import pickle  
import spacy
import json
import numpy as np
# import pandas as pd
import logging
import numpy
import nltk
from nltk.corpus import wordnet as wn
import os

import csv
from tqdm import tqdm_notebook as tqdm

nlp = spacy.load("en_core_web_sm")

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
sw = set(stopwords.words('english'))

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize('Eighty-seven miles to go, yet.  Onward!')

['Eighty', 'seven', 'miles', 'to', 'go', 'yet', 'Onward']

In [8]:
def load_dataset(filename):
    # read a winogrande style jsonl file
    # qID, sentence, option1, option2, answer
    this_set = []
    
    
    with open(filename,"r") as json_file:
        json_list = list(json_file)

        for json_str in json_list:
            result = json.loads(json_str)
            this_set.append(result)
    print("Loaded "+ filename + " with "+ str(len(this_set)) + " items.")
    return this_set

In [9]:
wsc = load_dataset("wsc.jsonl")

Loaded wsc.jsonl with 279 items.


In [10]:
all_sents = []
for i in range(len(wsc)):
    #print(wsc[i])
    all_sents.append(wsc[i]["sentence"])

In [13]:
# find the most frequent tokens
from collections import Counter

words = []

for sent in all_sents:
    temp_words = tokenizer.tokenize(sent.lower())
    for temp_word in temp_words:
        
        if temp_word not in sw:
            words.append(temp_word)
        #words.append(temp_word)

counter = Counter()
counter.update(words)
most_common = counter.most_common(100)
print(most_common)

[('_', 279), ('bill', 16), ('sam', 16), ('fred', 16), ('susan', 14), ('man', 14), ('john', 14), ('time', 14), ('put', 14), ('bob', 12), ('see', 11), ('years', 11), ('got', 11), ('gave', 11), ('tried', 10), ('tree', 10), ('work', 9), ('took', 9), ('saw', 9), ('get', 9), ('book', 9), ('paul', 8), ('tom', 8), ('much', 8), ('better', 8), ('one', 8), ('away', 8), ('would', 8), ('adam', 8), ('alice', 8), ('since', 8), ('knocked', 8), ('jane', 8), ('ago', 8), ('father', 8), ('george', 7), ('good', 7), ('though', 7), ('woman', 7), ('asked', 6), ('right', 6), ('table', 6), ('look', 6), ('lot', 6), ('jim', 6), ('pete', 6), ('full', 6), ('ann', 6), ('mark', 6), ('joe', 6), ('still', 6), ('started', 6), ('came', 6), ('mary', 6), ('library', 6), ('paid', 6), ('charlie', 6), ('home', 6), ('daughter', 6), ('carried', 6), ('door', 6), ('turned', 6), ('long', 6), ('passed', 6), ('boyfriend', 6), ('answer', 5), ('although', 5), ('police', 5), ('moved', 5), ('house', 5), ('education', 5), ('run', 5), ('l

In [14]:
# prune a type of dependency token
def prune_pos(text, target):
    doc = nlp(text)
    temp = []
    count = 0
    for token in doc:
        if token.text == "_":
            temp.append(token.text)
            
        elif token.dep_  in target:
            count = 1
            continue
        else:
            # print(token.dep_)
            temp.append(token.text)
            
    return " ".join(temp[:-1]) + ".", count

def generate_pos_set(all_sents, target):
    all_target = []
    
    for sent in all_sents:
        doc = nlp(sent)
        tars = [token.text for token in doc if token.pos_ == target]
        
        try:
            tars.remove("_")
        except:
            a = "nothing"
            
        all_target.extend(tars)
        
    return all_target

In [15]:
# all_POS = generate_pos_set(all_sents, "VERB")
# all_POS = generate_pos_set(all_sents, "PRON")
# all_POS = generate_pos_set(all_sents, "ADJ")
# all_POS = generate_pos_set(all_sents, "ADV")

all_POS = generate_pos_set(all_sents, "PROPN") # names
all_POS.append('Kamchatka')

In [16]:
print(all_POS)

['Joan', 'Susan', 'Joan', 'Susan', 'Paul', 'George', 'Paul', 'George', 'Frank', 'Bill', 'Frank', 'Bill', 'styrofoam', 'John', 'Billy', 'John', 'Billy', 'Tom', 'Ray', 'Tom', 'Ray', 'Sue', 'Sally', 'Sue', 'Sally', 'Sam', 'Tina', 'Sam', 'Tina', 'Anna', 'Lucy', 'Anna', 'Lucy', 'Frank', 'Tom', 'Frank', 'Tom', 'Jim', 'Kevin', 'Jim', 'Kevin', 'Pete', 'Martin', 'Pete', 'Martin', 'Susan', 'Ann', 'Susan', 'Ann', 'Sid', 'Mark', 'Sid', 'Mark', 'Susan', 'Ann', 'Susan', 'Ann', 'Joe', 'Joe', 'Mark', 'Mark', 'Ann', 'Mary', 'Ann', 'Mary', 'Bob', 'Charlie', 'Bob', 'Charlie', 'Bob', 'Charlie', 'Charlie', 'Bob', 'Charlie', 'Charlie', 'Bob', 'Adam', 'Adam', 'Bob', 'Adam', 'Adam', 'Adam', 'Bob', 'Bob', 'Adam', 'Bob', 'Bob', 'Sam', 'Sam', 'Anne', 'Anne', 'Alice', 'Alice', 'Jim', 'Jim', 'keyhole', 'keyhole', 'John', 'John', 'John', 'John', 'Bob', 'Carl', 'Bob', 'Carl', 'Sam', 'Amy', 'Amy', 'Sam', 'Amy', 'Amy', 'Mark', 'Pete', 'Pete', 'Mark', 'Pete', 'Pete', 'Joe', 'Thursday', 'Joe', 'Thursday', 'Paul', 'Paul'

In [20]:
# Pruning
pruned_wsc = []

# pruned_tokens = [" this ", " that "]
# pruned_tokens = [","]
# pruned_tokens = ["."]
# pruned_tokens = ["the", "The"]
# pruned_tokens = ["a", "A"]
# pruned_tokens = ["this", "that", "these", "those", "This", "That", "These", "Those"]
# pruned_tokens = ["has", "have", "had", "Has", "Have", "Had"]
# pruned_tokens = ["Because", "because"]
# pruned_tokens = ["go", "going", "went", "gone", "goes"]
# pruned_tokens = ["but", "But"]
# pruned_tokens = ["though", "although", "Though", "Although","since", "Since"] 
# pruned_tokens = ["but", "But", "and", "And"]
# pruned_tokens = ["very", "Very"]
# pruned_tokens = ["so", "So"]
# pruned_tokens = ["in", "In"]
# pruned_tokens = ["of", "Of"]
# pruned_tokens = ["on", "On"]
# pruned_tokens = ["with", "With"]
# pruned_tokens = ["and", "or", "but", "as", "because", "for", "just as", "neither", "nor", "not only", "so", "whether", "yet"]
# pruned_tokens.extend(["And", "Or", "But", "As", "Because", "For", "Just as", "Neither", "Nor", "Not only", "So", "Whether", "Yet"])



new_POS = []
for pos in all_POS:
    new_POS.append(pos.lower())
    
pruned_tokens = new_POS

num_case = []

for piece in wsc:
    new_piece = {}
    for key, value in piece.items():
        if key != "sentence":
            new_piece[key] = value
        else:
            temp = value.lower()
            if value[-1] != ".":
                value += "."
                
            temp_case = 0
            
            ###### uncomment this block if you want to prune candidates...###
            #################################################################
            # pruned_tokens=[]
            # cand1 = piece["option1"]
            # cand2 = piece["option2"]
            # pruned_tokens.extend(tokenizer.tokenize(cand1.lower()))
            # pruned_tokens.extend(tokenizer.tokenize(cand2.lower()))

            # names = []
            # for name in new_ADP:
            #     try:
            #         pruned_tokens.remove(name)
            #         names.append(name)
            #     except:
            #         a = "nothing"

            # # pruned_tokens = names
            # print(pruned_tokens)
            
            # if len(pruned_tokens) > 0:
                # temp_case = 1
            ################################################################
            
            for token in pruned_tokens:
                if temp.find(" "+ token + " ") != -1 or temp.find(token + " ") != -1 or temp.find( " "+token) != -1:
                    temp_case = 1

                temp = temp.replace(" "+ token + " "," ")
                temp = temp.replace(token + " "," ")
                temp = temp.replace(" " + token," ")
            
            # print(temp)
            new_piece[key] = temp
                
            num_case.append(temp_case)
            
            ###### uncomment this block if you want to prune by dep token ###
            #################################################################      
            
            # new_piece[key], temp_count=prune_pos(value.lower(), ["amod", "advmod", "npadvmod"])
            # new_piece[key], temp_count=prune_pos(value.lower(), ["neg"])
            # num_case.append(temp_count)
            
    pruned_wsc.append(new_piece)
                
print("Number of involved cases:",sum(num_case))

Number of involved cases: 184


In [21]:
def generate_sudo_set(cover_set, output_folder):
    # generate the seleted/unselected set
    with open(output_folder+"/dev.jsonl","w") as f1:
        with open(output_folder+"/wsc.jsonl", "w") as f2:
            for i,piece in enumerate(cover_set):
                # if i in selected_ids:
                f1.write(json.dumps(piece))
                f1.write("\n")
                f2.write(json.dumps(piece))
                f2.write("\n")

In [22]:
# the generated set could be used to evaluate winogrande
# generate_sudo_set(pruned_wsc, "./prune")